# 데이터 전처리

- 처리가 필요한 데이터
    + 누락 데이터 (NA) -> 누락 데이터 처리
    + 타입이 올바르지 않은 데이터 -> 데이터 타입 변경
    + 콤마, 공백, 모호한 대/소문자 사용 -> 콤마, 공백 제거, 대/소문자 변경
    + 의미없는 데이터 -> 의미있는 데이터 선별 및 제거

## NA의 종류

- NA == Not Available == Missing Data
- None, np.nan, pd.NaT
    + np.nan : float 타입으로 숫자의 missing 을 의미
    + pd.NaT : np.datetime64 타입으로 날짜의 missing 을 의미
    + np.inf : pd.options.mode.use_inf_as_na = True 를 하면 missing data 취급 / 기본값은 False 임
- pandas 에서의 연산은 기본적으로 NA 데이터를 제외하고 처리됨

In [1]:
import pandas as pd
import numpy as np
import shelve

In [2]:
pd.options.mode.use_inf_as_na = True

s = pd.Series([np.nan, pd.NaT, None, np.inf])
display(s)

print(s.count(), s.sum())

0     NaN
1     NaT
2    None
3     NaN
dtype: object

0 0


In [3]:
pd.options.mode.use_inf_as_na = False

s = pd.Series([np.nan, pd.NaT, None, np.inf])
display(s)

print(s.count(), s.sum())

0     NaN
1     NaT
2    None
3     inf
dtype: object

1 inf


## NA 데이터의 상등 비교

- None, np.inf 는 상등 비교가 가능함
- np.nan, pd.NaT는 상등 비교가 불가능함 (isna() 함수를 사용함)

In [4]:
print(f'None :   {None == None}',
      f'np.inf : {np.inf == np.inf}',
      f'np.nan : {np.nan == np.nan}',
      f'pd.NaT : {pd.NaT == pd.NaT}', sep="\n")

None :   True
np.inf : True
np.nan : False
pd.NaT : False


In [5]:
pd.options.mode.use_inf_as_na = True
s = pd.Series([np.nan, pd.NaT, None, np.inf])
s.isna()

0    True
1    True
2    True
3    True
dtype: bool

In [6]:
pd.options.mode.use_inf_as_na = False
s = pd.Series([np.nan, pd.NaT, None, np.inf])
s.isna()

0     True
1     True
2     True
3    False
dtype: bool

## NA 정보 확인

- `df.info()`
    + index, columns, dtypes, memory usage 정보 출력
    + 출력 정보의 정도를 조절할 수 있는 parameter 가 있음
    + memory_usage='deep', deep memory introspection 설정
    - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html#pandas-dataframe-info
- `df.isna()`
    + Boolean 데이터로 작성된 DataFrame 객체 반환 (NA value -> True)
    + isna, isnull 은 동일 동작 (isnull 은 isna 의 alias)
    + any(), all() 등으로 정보를 요약할 수 있음
    + `df.isna().any()`
    + `df.isna().all()`
    + `df.isna().any().any()` : DataFrame 에 하나라도 NA가 있으면 True
    + https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html#pandas-dataframe-isna
- `df.any(axis=0)`
    + column 별로 값이 하나라도 True 인 경우 True 아니면 False
    + axis=1 로 하면 row 별로 확인
- `df.all(axis=0)`
    + column 별로 모든 값이 True 인 경우 True 아니면 False
    + axis=1 로 하면 row 별로 확인    

In [7]:
df = pd.read_csv('./data/easySample2.csv')
df.head()

,ID,pname,birth,dept,english,japanese,chinese,salary,overtime
0,18030201,James Kim,1990-01-23,Education,1,1.0,,"3,456",23:10:10
1,18030202,Rose Hwang,1992-10-11,Marketing,,2.0,NaN,"4,320",10:15:17
2,19030401,Sam Park,1995-07-02,Education,1,NaN,NaN,"5,600",16:21:10
3,19070101,Chris Jang,1990-11-23,Education,NaN,NaN,3,"4,500",15:00:20
4,19070102,Grace Lee,1993-02-01,Marketing,NaN,NaN,NaN,"3,150",21:19:50


In [8]:
df.info()
# 하기 정보 중 RangeIndex: 10 entries, 0 to 9 을 통하여 전체 10개의 Entry 가 있음을 알 수 있음
# 4   english   6 non-null      object 
# 5   japanese  5 non-null      float64
# 6   chinese   6 non-null      object 
# 상기 3개의 Column 의 경우 10개의 데이터가 안되므로 NA 데이터가 있다고 판단할 수 있음
# 특히 english 의 경우 object dtype 이므로 데이터 중 공백이 포함되어 있음을 식별할 수 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        10 non-null     int64  
 1   pname     10 non-null     object 
 2   birth     10 non-null     object 
 3   dept      10 non-null     object 
 4   english   6 non-null      object 
 5   japanese  5 non-null      float64
 6   chinese   6 non-null      object 
 7   salary    10 non-null     object 
 8   overtime  10 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 848.0+ bytes


In [9]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        10 non-null     int64  
 1   pname     10 non-null     object 
 2   birth     10 non-null     object 
 3   dept      10 non-null     object 
 4   english   6 non-null      object 
 5   japanese  5 non-null      float64
 6   chinese   6 non-null      object 
 7   salary    10 non-null     object 
 8   overtime  10 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 4.4 KB


In [10]:
df1 = df[['english', 'japanese', 'chinese']]
df1.isna()

,english,japanese,chinese
0,False,False,False
1,False,False,True
2,False,True,True
3,True,True,False
4,True,True,True
5,True,True,False
6,False,True,True
7,False,False,False
8,False,False,False
9,True,False,False


In [11]:
df1.isna().any()

english     True
japanese    True
chinese     True
dtype: bool

In [12]:
df1.isna().any(axis=1)

0    False
1     True
2     True
3     True
4     True
5     True
6     True
7    False
8    False
9     True
dtype: bool

In [13]:
df1.isna().all()

english     False
japanese    False
chinese     False
dtype: bool

In [14]:
df1.isna().all(axis=1)

0    False
1    False
2    False
3    False
4     True
5    False
6    False
7    False
8    False
9    False
dtype: bool

In [15]:
df1.isna().any().any()

True

## NA 제거

- `df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)`
- axis
    + axis=0 이거나 'index' : NA value 포함 행(row) 제거
    + axis=1 이거나 'columns' : NA value 포함 열(column) 제거    
- how
    + how='any' : NA value 가 하나라도 포함된 경우 True
    + how='all' : 모든 값이 NA value 인 경우 True    
- thresh
    + int, non-NA value 갯수가 설정 값 이상일 떄 제거 안 함
- subset
    + array-like, NA value 를 살펴볼 label 목록
    + axis=0 : columns 에 대한 label 을 목록으로 작성함
- inplace
    + bool, True 인 경우 대상에 직접 반영하고, None 을 반환

In [16]:
df = pd.read_csv('./data/easySample2.csv')
df.head()

,ID,pname,birth,dept,english,japanese,chinese,salary,overtime
0,18030201,James Kim,1990-01-23,Education,1,1.0,,"3,456",23:10:10
1,18030202,Rose Hwang,1992-10-11,Marketing,,2.0,NaN,"4,320",10:15:17
2,19030401,Sam Park,1995-07-02,Education,1,NaN,NaN,"5,600",16:21:10
3,19070101,Chris Jang,1990-11-23,Education,NaN,NaN,3,"4,500",15:00:20
4,19070102,Grace Lee,1993-02-01,Marketing,NaN,NaN,NaN,"3,150",21:19:50


In [17]:
# Regular Expression 을 통하여 White Space 를 np.nan 으로 변경
df = df.replace(r'^\s+$', np.nan, regex=True)

In [18]:
df = df.iloc[:3, 3:7]
df

,dept,english,japanese,chinese
0,Education,1,1.0,NaN
1,Marketing,NaN,2.0,NaN
2,Education,1,NaN,NaN


In [19]:
# NA 가 아닌 것을 확인 == ~df.isna()
df.notna()

,dept,english,japanese,chinese
0,True,True,True,False
1,True,False,True,False
2,True,True,False,False


In [20]:
# NA 가 하나라도 포함된 열 제거
df.dropna(axis=1)

,dept
0,Education
1,Marketing
2,Education


In [21]:
# NA 가 하나라도 포함된 행 제거
df.dropna()

,dept,english,japanese,chinese


In [22]:
# 각 컬럼에 대해 모든 항이 NA 인 경우 제거
df.dropna(axis=1, how='all')

,dept,english,japanese
0,Education,1,1.0
1,Marketing,NaN,2.0
2,Education,1,NaN


In [23]:
# NA 가 아닌 값이 3개 이상인 행에 대해 제거 안 함
df.dropna(thresh=3)

,dept,english,japanese,chinese
0,Education,1,1.0,NaN


In [24]:
# dropna 의 기준을 english 와 japanese 로 제한하며, 1개라도 NA 가 있는 행을 제거
df.dropna(how='any', subset=['english', 'japanese'])

,dept,english,japanese,chinese
0,Education,1,1.0,NaN


In [25]:
# 1개라도 NA 가 있는 행을 제거해서 df 자체를 수정
df.dropna(inplace=True)
df

,dept,english,japanese,chinese


## NA 채우기

- `df.fillna(value=None, method=None, axis=None, inplace=False, limit=None, ...)`
- NA value 를 value 또는 method 를 사용하여 변경한 DataFrame 객체 반환
- value
    + scalar, dit, Series, DataFrame
    + NA values 를 대신할 값을 지정함
    + dict, Series, DataFrame 을 사용해 행/열 별 채우기 값 별도 지정 가능
- method
    + {'backfill', 'bfill', 'pad', 'ffill', None}
    + value= None 일 때, NA values 를 대신할 값 선정 방법을 지정함
    + 'backfill', 'bfill' : [아래 -> 위]로 올라가면서 다음 발견되는 valid observation 으로 채움
    + 'pad', 'ffill' : [위 -> 아래]로 내려가면서 이전에 발견된 valid observation 으로 채움
- asis
    + axis= 0 이거나 'index' : 행 방향으로 채우기 진행
    + axis= 1 이거나 'columns' : 열 방향으로 채우기 진행    
- inplace
    + bool, True 인 경우 대상에 직접 반영하고, None 을 반환
- limit
    + NA values 를 다른 value 로 변경하는 동작의 최대 횟수

In [26]:
df = pd.read_csv("./data/easySample2.csv")
df = df.replace(r'^\s+$', np.nan, regex=True)

# index 는 indexing 에 포함되지 않음
df = df.iloc[:6, [4,6]]
df.iloc[0,1] = 2
df.iloc[1,1] = 3
df

,english,chinese
0,1,2
1,NaN,3
2,1,NaN
3,NaN,3
4,NaN,NaN
5,NaN,1


In [27]:
# 모든 NA 를 0 으로 채움
df.fillna(value=0)

,english,chinese
0,1,2
1,0,3
2,1,0
3,0,3
4,0,0
5,0,1


In [28]:
# english 의 NA 는 -1, chinese 의 NA 는 -2 로 채움
df.fillna(value={"english": -1, "chinese": -2})

,english,chinese
0,1,2
1,-1,3
2,1,-2
3,-1,3
4,-1,-2
5,-1,1


In [29]:
# NA 를 채울 때 이전에 발견된 값으로 채우는데 2개까지만 채움
df.fillna(method='ffill', limit=2)

,english,chinese
0,1,2
1,1,3
2,1,3
3,1,3
4,1,3
5,NaN,1


In [30]:
# NA를 채울 때 다음 발견되는 값으로 채움
df.fillna(method='backfill')

,english,chinese
0,1,2
1,1,3
2,1,3
3,NaN,3
4,NaN,1
5,NaN,1


## Value 대체

- `df.replace(to_replace, value=None, inplace=False, limit=None, regex=False, ...)`
- to_replace 로 주어진 대상이 value 로 주어진 값으로 변경된 DataFrame 객체
- to_replace
    + str, regex, list, dict, Series, int, float, None
    + value 로 대체될 값들을 찾는 방법
    + API 에서 상세 설명 참조
      https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html#pandas-dataframe-replace
- value
    + scalar, dict, list, str, regex, default None
    + to_replace 에 매칭되는 값을 대체할 값을 지정
    + dict 을 사요하여 열 별 채우기 값을 별도 지정 가능
- inplace
    + bool, True 인 경우 대상에 직접 반영하고, None 을 반환
- limit
    + NA values 를 다른 value 로 변경하는 동작의 최대 횟수
- regex
    + bool or same types as to_replace
    + True 설정 시 to_replace 및 value 의 정규식 사용 가능
    + to_replace 는 str 을 사용해야 함

In [31]:
df = pd.read_csv("./data/easySample2.csv")
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df[['dept']]
df

,dept
0,Education
1,Marketing
2,Education
3,Education
4,Marketing
5,Education
6,Accounting
7,Sales
8,Sales
9,Education


In [32]:
# Education 을 E 로 변경
df.replace('Education', value='E')

,dept
0,E
1,Marketing
2,E
3,E
4,Marketing
5,E
6,Accounting
7,Sales
8,Sales
9,E


In [33]:
# Education, Marketing, Sales, Accounting 을 각각 E, M, S, A 로 변경
df.replace(to_replace=['Education', 'Marketing', 'Sales', 'Accounting'], 
           value=['E', 'M', 'S', 'A'])

,dept
0,E
1,M
2,E
3,E
4,M
5,E
6,A
7,S
8,S
9,E


In [34]:
temp = df['dept'].unique()
v = [ x[0] for x in temp ]
df.replace(temp, v)

,dept
0,E
1,M
2,E
3,E
4,M
5,E
6,A
7,S
8,S
9,E


In [35]:
# Education, Marketing, Sales, Accounting 을 각각 0,1,2,3 로 변경
df.replace(to_replace=['Education', 'Marketing', 'Sales', 'Accounting'], 
           value=['E', 'M', 'S', 'A'])

,dept
0,E
1,M
2,E
3,E
4,M
5,E
6,A
7,S
8,S
9,E


In [36]:
# dictionary 사용 시 { to_replace: value } 구조
df.replace({'Education': 0, 'Marketing': 1, 'Sales': 2, 'Accounting': 3})

,dept
0,0
1,1
2,0
3,0
4,1
5,0
6,3
7,2
8,2
9,0


In [37]:
# Series 사용 시 to_replace 가 index 인 구조
dept_list = df['dept'].unique()
value_list = np.array(range(len(dept_list)))
s1 = pd.Series(value_list, index=dept_list)

df.replace(s1)

,dept
0,0
1,1
2,0
3,0
4,1
5,0
6,2
7,3
8,3
9,0


# DataFrame 합치기

## pd.concat

- 여러 개의 데이터프레임 하나로 합치기
- `pd.concat(objs, axis=0, join='outer', ignore_index=False, verify_integrity=False, ...)`
- index 를 기준으로 행/열 방향으로 DataFrame 을 병합
- objs : a sequence or mapping of Series or DataFrame objects
- axis : 0 or 'index' -> 행 방향, 1 or 'columns' -> 열 방향 
- join : {'outer', 'inner'}, 매치되는 index/columns 없을 때의 동작
    + outer : NaN 채우기
    + inner : 삭제하기
- ignore_index : index 를 무시하고 RangeIndex 로 변경
- verify_integrity : True -> 중복 데이터 있으면 오류 발생
- https://pandas.pydata.org/docs/reference/api/pandas.concat.html#pandas-concat

In [38]:
import pandas as pd
import numpy as np

In [39]:
df1 = pd.DataFrame({'A': np.arange(1, 5), 'B': list('opqr')}, index=list('abcd'))
df2 = pd.DataFrame({'A': [5, 6, 7], 'B': list('stu')}, index=list('efg'))
df3 = pd.DataFrame({'C': [10, 20, 15, 40], 'D': list('QXYZ')}, index=list('abcd'))
df4 = pd.DataFrame({'A': [1, 20, 15], 'B': list('xyz')}, index=list('abc'))

display(df1, df2, df3, df4)

,A,B
a,1,o
b,2,p
c,3,q
d,4,r


,A,B
e,5,s
f,6,t
g,7,u


,C,D
a,10,Q
b,20,X
c,15,Y
d,40,Z


,A,B
a,1,x
b,20,y
c,15,z


In [40]:
pd.concat([df1, df2])

,A,B
a,1,o
b,2,p
c,3,q
d,4,r
e,5,s
f,6,t
g,7,u


In [41]:
pd.concat([df1, df3], axis=1)

,A,B,C,D
a,1,o,10,Q
b,2,p,20,X
c,3,q,15,Y
d,4,r,40,Z


In [42]:
pd.concat([df1, df3])

,A,B,C,D
a,1.0,o,NaN,NaN
b,2.0,p,NaN,NaN
c,3.0,q,NaN,NaN
d,4.0,r,NaN,NaN
a,NaN,NaN,10.0,Q
b,NaN,NaN,20.0,X
c,NaN,NaN,15.0,Y
d,NaN,NaN,40.0,Z


In [43]:
pd.concat([df1, df4], ignore_index=True, verify_integrity=True)

,A,B
0,1,o
1,2,p
2,3,q
3,4,r
4,1,x
5,20,y
6,15,z


In [44]:
pd.concat([df1, df4], axis=1, join='outer')

,A,B,A,B
a,1,o,1.0,x
b,2,p,20.0,y
c,3,q,15.0,z
d,4,r,NaN,NaN


In [45]:
pd.concat([df1, df4], axis=1, join='inner')

,A,B,A,B
a,1,o,1,x
b,2,p,20,y
c,3,q,15,z


## pd.merge

- `pd.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False. ...)`
- on 에 지정된 병합 기준 또는 index 에 따라 left, right 병합
- left, right : DataFrame or Named Series
- how : {'left','right','outer','inner'}, default = 'inner'
- on : label or list, 병합 기준 지정 (columns or index level names)
- left_on, right_on : label or list, 왼쪽 / 오른쪽 병합 기준 지정
- left_index, right_index 
    + True / False 를 사용하야 index 를 병합 기준으로 사용할지 여부 지정
    + columns 가 다를 경우 True 로 지정하여야 함
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html#pandas-dataframe-merge

In [46]:
df1 = pd.DataFrame({'A': [1,20,15,5],   'B': list('XYZQ')}, index=list('abce'))
df2 = pd.DataFrame({'C': [10,20,15,40], 'B': list('XYMN')}, index=list('abcd'))
df3 = pd.DataFrame({'C': [10,20,15,40], 'D': list('XYMN')}, index=list('abcd'))
df4 = pd.DataFrame({'C': [10,20,15,40], 'D': list('QXYZ')}, index=list('abcd'))

display(df1, df2)

,A,B
a,1,X
b,20,Y
c,15,Z
e,5,Q


,C,B
a,10,X
b,20,Y
c,15,M
d,40,N


In [47]:
pd.merge(left=df1, right=df2, on='B')

,A,B,C
0,1,X,10
1,20,Y,20


In [48]:
display(df1, df2)

,A,B
a,1,X
b,20,Y
c,15,Z
e,5,Q


,C,B
a,10,X
b,20,Y
c,15,M
d,40,N


In [49]:
pd.merge(left=df1, right=df2, on='B', how='outer')

,A,B,C
0,1.0,X,10.0
1,20.0,Y,20.0
2,15.0,Z,NaN
3,5.0,Q,NaN
4,NaN,M,15.0
5,NaN,N,40.0


In [50]:
pd.merge(left=df1, right=df2, on='B', how='left')

,A,B,C
0,1,X,10.0
1,20,Y,20.0
2,15,Z,NaN
3,5,Q,NaN


In [51]:
pd.merge(left=df1, right=df2, on='B', how='right')

,A,B,C
0,1.0,X,10
1,20.0,Y,20
2,NaN,M,15
3,NaN,N,40


In [52]:
display(df1, df3)

,A,B
a,1,X
b,20,Y
c,15,Z
e,5,Q


,C,D
a,10,X
b,20,Y
c,15,M
d,40,N


In [53]:
pd.merge(left=df1, right=df3, left_on='B', right_on='D')

,A,B,C,D
0,1,X,10,X
1,20,Y,20,Y


In [54]:
display(df1.head(), df4.head())

,A,B
a,1,X
b,20,Y
c,15,Z
e,5,Q


,C,D
a,10,Q
b,20,X
c,15,Y
d,40,Z


In [55]:
pd.merge(left=df1, right=df4, left_index=True, right_index=True)

,A,B,C,D
a,1,X,10,Q
b,20,Y,20,X
c,15,Z,15,Y


# 데이터 삭제 - pd.drop

- `x.drop(labels, axis=0, ...)`
    + labels : 한 개의 label 또는 list-like index / column labels
    + axis=0 or 'index' : 행 삭제
    + axis-1 or 'columns' : 열 삭제
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html#pandas-dataframe-drop

![nn](./images/pandas-04.png)

In [56]:
s = pd.Series([10,13,15], index=list('ABB'))
s1 = s.drop('B')
display(s, s1)

A    10
B    13
B    15
dtype: int64

A    10
dtype: int64

In [57]:
df = pd.DataFrame({'a': [10,13], 'b': [1,5], 'c': [20,5]}, index=list('AB'))
df1 = df.drop(['a', 'b'], axis=1)
display(df, df1)

,a,b,c
A,10,1,20
B,13,5,5


,c
A,20
B,5


# 데이터 추가 - pd.append

- `x.append(other, ignore_index=FAlse, verify_integrity=False, ...)`
    + other 에 전달된 데이터를 추가한 객체 반환
    + other
        - x is DataFrame : DataFrame, Series/dict-like, list of these
        - x is Series : Series or list/tuple of Series
    + ignore_index : True -> index labels 사용하지 ㅇ낳음, index 없는 대상 추가 시 필수
    + verify_integrity : True -> index 중복 시 ValueError 현상
    + https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html#pandas-dataframe-append
- 향후 버전에서 지원 안하므로 `pd.concat()` 사용 권장

In [58]:
df1 = pd.DataFrame({'a': [10,13], 'b': [1,5]}, index=['A','B'])
df2 = pd.DataFrame({'a': [1,3],   'b': [4,8]}, index=['C','D'])

display(df1, df2)

,a,b
A,10,1
B,13,5


,a,b
C,1,4
D,3,8


In [59]:
df3 = df1.append(df2)
df3

/var/folders/3k/bgnrnfc15w792712mgb6g0_c0000gn/T/ipykernel_11792/3362903225.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df1.append(df2)


,a,b
A,10,1
B,13,5
C,1,4
D,3,8


In [60]:
df1 = pd.DataFrame({'a': [10,13], 'b': [1,5]}, index=['A','B'])
df1

,a,b
A,10,1
B,13,5


In [61]:
df2 = df1.append({'a': 1, 'b':4}, ignore_index=True)
df2

/var/folders/3k/bgnrnfc15w792712mgb6g0_c0000gn/T/ipykernel_11792/4145202659.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df1.append({'a': 1, 'b':4}, ignore_index=True)


,a,b
0,10,1
1,13,5
2,1,4


# 데이터 변환

## Series.map

- `s.map(arg, na_action=None)`
    + arg 로 전달된 내용이 각 항에 적용된 Series 반환
- arg : function, dict, Series
    + Series 의 각 항에 적용될 내용
    + dict 가 사용될 경우 key 에 없는 것이 Series 에 포함되어 있으면 NaN이 됨
- na_action : {None, 'ignore'} (default: None)
    + ignore : NA value 에 대해 동작을 무시하고 NaN, None, NaT 로 채움
- API: https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html#pandas-series-map

![nn](./images/pandas-05.png)

In [62]:
s = pd.Series(['Park', 'Kim', np.nan, 'Lee', None, pd.NaT])

In [63]:
'Mr. ' + s 

0    Mr. Park
1     Mr. Kim
2         NaN
3     Mr. Lee
4         NaN
5         NaN
dtype: object

In [64]:
s.map('Mr. {}'.format)

0    Mr. Park
1     Mr. Kim
2     Mr. nan
3     Mr. Lee
4    Mr. None
5     Mr. NaT
dtype: object

In [65]:
s.map('Mr. {}'.format, na_action='ignore')

0    Mr. Park
1     Mr. Kim
2         NaN
3     Mr. Lee
4        None
5         NaT
dtype: object

In [66]:
s = pd.Series([10,10,30,10,20])

In [67]:
# 처리 방법이 명기되지 않은 데이터는 NaN
s.map({10:0, 20:1})

0    0.0
1    0.0
2    NaN
3    0.0
4    1.0
dtype: float64

In [68]:
# 처리 방법이 명기되지 않은 데이터는 그대로 유지
s.replace({10:0, 20:1})

0     0
1     0
2    30
3     0
4     1
dtype: int64

### 성적처리

In [69]:
s = pd.Series([75,66,120,80,100,95,50], name='score')
df = pd.DataFrame(s)
df.index.name = 'number'
df

,score
number,
0,75
1,66
2,120
3,80
4,100
5,95
6,50


In [70]:
# df에 'grade'라는 column을 추가한다
# 'grade'의 values는 'df.score'를 사용하여 구하며, 세부조건은 다음과 같다
# 0~100 점 사이의 데이터가 아닌 경우 'Error'
# 100~90 : 'A', 89~80 : 'B', 79~70 : 'C', 69~60 : 'D', 59~0 : 'F' 
def calculate_grade(score):
    grade = list('FFFFFFDCBAA')
    return grade[score // 10] if -1 < score < 101 else 'Error'

#df['grade'] = df['score'].map(calculate_grade)
df['grade'] = df['score'].map(lambda score: 'FFFFFFDCBAA'[score // 10] if 0<= score <=100 else 'Error')
df

,score,grade
number,,
0,75,C
1,66,D
2,120,Error
3,80,B
4,100,A
5,95,A
6,50,F


### 부서별 코드 번호 부여

In [71]:
import shelve

with shelve.open("./data/mysample") as data :
  df = data['sample3']
  df = df.loc[:, ['pname', 'dept', 'salary', 'overtime']]

df

,pname,dept,salary,overtime
ID,,,,
18030201,James Kim,Education,3456,0 days 23:10:10
18030202,Rose Hwang,Marketing,4320,0 days 10:15:17
19030401,Sam Park,Education,5600,0 days 16:21:10
19070101,Chris Jang,Education,4500,0 days 15:00:20
19070102,Grace Lee,Marketing,3150,0 days 21:19:50
19070103,Juile Yoon,Education,4200,0 days 14:10:40
19080101,Chirle Song,Accounting,4800,0 days 09:50:30
19080102,Bob Kim,Sales,10100,0 days 08:40:40
19090201,John Park,Sales,6840,0 days 17:30:20


In [72]:
# df에 'dept_code'라는 column을 추가한다
# 'dept_code'는 dept에 1부터 시작하여 1씩 증가하는 일련번호를 부여한 것이다
# 단, 부서명의 알파벳순(오름차순)에 따라 번호가 부여된다
dept_list = sorted(df['dept'].unique())
dept_code = dict(zip(dept_list, range(1, len(dept_list)+1)))

df['dept_code'] = df['dept'].map(dept_code)
df

,pname,dept,salary,overtime,dept_code
ID,,,,,
18030201,James Kim,Education,3456,0 days 23:10:10,2
18030202,Rose Hwang,Marketing,4320,0 days 10:15:17,3
19030401,Sam Park,Education,5600,0 days 16:21:10,2
19070101,Chris Jang,Education,4500,0 days 15:00:20,2
19070102,Grace Lee,Marketing,3150,0 days 21:19:50,3
19070103,Juile Yoon,Education,4200,0 days 14:10:40,2
19080101,Chirle Song,Accounting,4800,0 days 09:50:30,1
19080102,Bob Kim,Sales,10100,0 days 08:40:40,4
19090201,John Park,Sales,6840,0 days 17:30:20,4


### overtime 금액 계산

In [73]:
# df의 'overtime'에 대해서 금액을 산정해 salary에 더한다
# overtime 계산은 1일 100, 1시간 5로 계산한다
# 단 overtime이 10시간 미만인 경우는 금액을 지불하지 않는다
# salary에 더한 뒤에는 overtime을 0으로 변경한다
with shelve.open("./data/mysample") as data :
  df = data['sample3']
  df = df.loc[:, ['pname', 'dept', 'salary', 'overtime']]

def func(x):
    hours = x.seconds // 3600
    if x.days == 0 and hours < 10:
        return 0
    return x.days * 100 + hours * 5
    
# df['salary'] += df['overtime'].map(func)
df['salary'] += df['overtime'].map(lambda x : x.days * 100 + (x.seconds // 3600) * 0 if (x.seconds // 3600) < 10 else 5)
df['overtime'] = 0

df

,pname,dept,salary,overtime
ID,,,,
18030201,James Kim,Education,3461,0
18030202,Rose Hwang,Marketing,4325,0
19030401,Sam Park,Education,5605,0
19070101,Chris Jang,Education,4505,0
19070102,Grace Lee,Marketing,3155,0
19070103,Juile Yoon,Education,4205,0
19080101,Chirle Song,Accounting,4800,0
19080102,Bob Kim,Sales,10100,0
19090201,John Park,Sales,6845,0


## apply

- `x.apply(func, axis, ..., args=(), **kwds)`
    + x : DataFrame, Series
- axis 설정에 따라 행/열 별로 func 에 주어진 함수를 적용한 결과를 반환
- func : 각 행이나 열에 적용할 함수
    + 함수는 lambda 로 작성하거나 numpy, Series 등에서 제공되는 것 사용
    + apply 는 행/열에 함수를 저굥ㅇ함 (map은 각 항에 함수를 적용)
- axis : 0 or 'index' : 각 column 에 적용, 1 or 'columns' : 각 row 에 적용
- args : array 또는 Series 를 포함한 tuple 로 작성
    + func 에 전달할 Positional Arguments
- kwds 
    + func 에 전달할 Keyword Arguments
- API : https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html#pandas-dataframe-apply
- aggregating functions (13개) 를 제외한 다른 함수는 transform function 이라고 함
- aggregating functions 와 transform functions 는 묶어서 리스트 형태로 apply 에 전달 불가</b>

### 13 aggregating functions available in Pandas
- apply 사용 시 묶어서 사용할 수 있는 함수 (aggregating functions)
    - mean() : Compute mean of groups
    - sum() : Compute sum of group values
    - size() : Compute group sizes
    - count() : Compute count of group
    - std() : Standard deviation of groups
    - var() : Compute variance of groups
    - sem() : Standard error of the mean of groups
    - describe() : Generate descriptive statstics
    - first() : Compute first of group values
    - last() : Compute last of group values
    - nth() : Take nth value, or subset if n is a list
    - min() : Compute min of group values
    - max() : Compute max of group values

In [74]:
import pandas as pd
import numpy as np

np.random.seed(1000)
df = pd.DataFrame(np.random.randint(50, 100, (4,3)), columns=list('ABC'))
df

,A,B,C
0,73,57,50
1,80,78,51
2,50,75,95
3,90,91,78


In [75]:
df.sum()

A    293
B    301
C    274
dtype: int64

In [76]:
df.apply(sum)

A    293
B    301
C    274
dtype: int64

In [77]:
df.apply(min, axis=1)

0    50
1    51
2    50
3    78
dtype: int64

In [78]:
df.apply([sum, min, max])

,A,B,C
sum,293,301,274
min,50,57,50
max,90,91,95


In [79]:
df.apply([sum, min, max], axis=1)

,sum,min,max
0,180,50,73
1,209,51,80
2,220,50,95
3,259,78,91


In [80]:
import shelve

df = shelve.open('./data/mysample')['sample3']
df = df.loc[:, ['english', 'chinese', 'japanese']]
df

,english,chinese,japanese
ID,,,
18030201,1,0,1
18030202,0,0,2
19030401,1,0,0
19070101,0,3,0
19070102,0,0,0
19070103,0,1,0
19080101,2,0,0
19080102,1,1,1
19090201,3,2,1


In [81]:
# ID 별로 가지고 있는 어학 등급의 갯수
df.apply(np.count_nonzero, axis=1) # Transform

ID
18030201    2
18030202    1
19030401    1
19070101    1
19070102    0
19070103    1
19080101    1
19080102    3
19090201    3
19090202    2
dtype: int64

In [82]:
# 어학 종류별 등급의 갯수
df.apply(pd.Series.value_counts)

,english,chinese,japanese
0,5,5,5
1,3,3,3
2,1,1,1
3,1,1,1


# 데이터의 그룹별 작업

- 작업 목적에 따른 분류

1. Aggregation
    + 각 그룹에 함수 적용 후 그룹별 함수 결과 형태의 객체 반환
    + 예) 그룹별 합계, 평균, 갯수 구하기
2. Transformation
    + 각 그룹에 함수 적용 후, index-like 객체 반환
    + 예) 그룹 내 데이터 표준화, 각 그룹별 산출값으로 NA value 채우기
3. Filtration
    + 각 그룹에 함수 적용 후 그 결과가 True 인 것만 남김 (False인 것 삭제)
    + 예) 데이터 갯수가 적은 그룹 제거, 합계, 평균 등에 기반한 데이터 추출

In [83]:
# Aggregation:     dept별 salary 평균 : index 가 변경됨
# Transformation : dept별 salary 평균 : index 가 유지됨
# Filtration :     dept별 salary 평균이 전체 평균 초과 : index 가 유지되나 filter 되어 표시됨

![nn](./images/pandas-06.png)

## groupby 작업

- `df.groupby(by=None, axis=0, level=None, sort=True, as_index=True, ...)`
- by/level 에 의해 그룹화된 DataFrameGroupBy / SeriesGroupBy 객체 반환
- by : mapping, function, label or list of labels
    + functions 의 경우 객체의 index 각 항을 대상으로 함
- axis : 0 인 경우 행, 1 인 경우 열 기준으로 그룹 나누기 작업 진행
- level : Multiindex 인 경우 level 을 기준으로 그룹 나누기
- sort : 정렬할 것인지 결정하는 것으로 False 가 성능면에서 좋음
- as_index : True 인 경우 group_label 을 index 로 사용함
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html#pandas-dataframe-groupby

In [84]:
def makeSample4():
    mySample = shelve.open('./data/mysample')
    df = mySample['sample3']
    df['dept'] = df['dept'].astype(np.object_)
    df['age'] = df['birth'].map(lambda x : 2023 - x.year + 1)
    df['gender'] = pd.Series(np.array(list('MFMMFFMMMF')), index=df.index)
    df.index = pd.RangeIndex(len(df))
    mySample['sample4'] = df
    print("Done!!! make sample4")
    mySample.close()
    return df

df = makeSample4()

Done!!! make sample4


In [85]:
with shelve.open('./data/mysample') as myf:
    df = myf['sample4']

df.head()

,pname,birth,dept,english,japanese,chinese,salary,overtime,age,gender
0,James Kim,1990-01-23,Education,1,1,0,3456,0 days 23:10:10,34,M
1,Rose Hwang,1992-10-11,Marketing,0,2,0,4320,0 days 10:15:17,32,F
2,Sam Park,1995-07-02,Education,1,0,0,5600,0 days 16:21:10,29,M
3,Chris Jang,1990-11-23,Education,0,0,3,4500,0 days 15:00:20,34,M
4,Grace Lee,1993-02-01,Marketing,0,0,0,3150,0 days 21:19:50,31,F


In [86]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype          
---  ------    --------------  -----          
 0   pname     10 non-null     object         
 1   birth     10 non-null     datetime64[ns] 
 2   dept      10 non-null     object         
 3   english   10 non-null     int32          
 4   japanese  10 non-null     int32          
 5   chinese   10 non-null     int32          
 6   salary    10 non-null     int32          
 7   overtime  10 non-null     timedelta64[ns]
 8   age       10 non-null     int64          
 9   gender    10 non-null     object         
dtypes: datetime64[ns](1), int32(4), int64(1), object(3), timedelta64[ns](1)
memory usage: 768.0+ bytes


In [87]:
# df.groupby 메서드를 사용하여 'dept'를 기준으로 DataFrameGroupBy 객체 생성함
dept_gb = df.groupby(by='dept')
type(dept_gb), dept_gb

(pandas.core.groupby.generic.DataFrameGroupBy,
 <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7f8471aeff10>)

In [88]:
df1 = df[['pname', 'dept', 'salary', 'overtime', 'age', 'gender']]
# df1을 사용하여 작업해 보자
# df1을 사용하여 'dept'를 기준으로 DataFrameGroupBy 객체를 생성하고 이름을 dept_gb로 한다
# dept_gb를 사용하여 각 그룹의 이름과, 그룹별 DataFrame을 출력해 보자
# DataFrameGroupBy 객체는 Iterable 이다

# dept_gb = df1.groupby(by='dept')
# temp = next(iter(dept_gb))
# name, df_part = temp
# display(type(name), type(df_part))

for name, dftemp in df1.groupby(by='dept'):
    display(name, dftemp)
    print("-" * 60)

'Accounting'

,pname,dept,salary,overtime,age,gender
6,Chirle Song,Accounting,4800,0 days 09:50:30,31,M


------------------------------------------------------------


'Education'

,pname,dept,salary,overtime,age,gender
0,James Kim,Education,3456,0 days 23:10:10,34,M
2,Sam Park,Education,5600,0 days 16:21:10,29,M
3,Chris Jang,Education,4500,0 days 15:00:20,34,M
5,Juile Yoon,Education,4200,0 days 14:10:40,32,F
9,Anne Lee,Education,4750,0 days 19:50:20,31,F


...

,pname,dept,salary,overtime,age,gender
1,Rose Hwang,Marketing,4320,0 days 10:15:17,32,F
4,Grace Lee,Marketing,3150,0 days 21:19:50,31,F


------------------------------------------------------------


'Sales'

,pname,dept,salary,overtime,age,gender
7,Bob Kim,Sales,10100,0 days 08:40:40,33,M
8,John Park,Sales,6840,0 days 17:30:20,32,M


------------------------------------------------------------


In [89]:
# 위에서 생성된 dept_gb를 사용하여 'Education'에 대한 DataFrame만 출력해 보자
# DataFrameGroupBy.get_group('group_name') 사용

temp = df1.groupby(by='dept').get_group('Education')
display(type(temp), temp)

pandas.core.frame.DataFrame

,pname,dept,salary,overtime,age,gender
0,James Kim,Education,3456,0 days 23:10:10,34,M
2,Sam Park,Education,5600,0 days 16:21:10,29,M
3,Chris Jang,Education,4500,0 days 15:00:20,34,M
5,Juile Yoon,Education,4200,0 days 14:10:40,32,F
9,Anne Lee,Education,4750,0 days 19:50:20,31,F


In [90]:
# 그룹별 작업은 다음 단계를 거쳐 결과를 만들어 낸다
# 1. Splitting : 그룹 분류 기준에 따라 데이터를 그룹으로 분리
# 2. Applying : 각 그룹별로 연산 적용
# 3. Combining : applying의 결과를 하나의 데이터 구조로 결합

- Aggregation 작업
![nn](./images/pandas-07.png)

In [91]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype          
---  ------    --------------  -----          
 0   pname     10 non-null     object         
 1   dept      10 non-null     object         
 2   salary    10 non-null     int32          
 3   overtime  10 non-null     timedelta64[ns]
 4   age       10 non-null     int64          
 5   gender    10 non-null     object         
dtypes: int32(1), int64(1), object(3), timedelta64[ns](1)
memory usage: 568.0+ bytes


In [92]:
# df1을 사용하여 
#'dept' 별로 group를 만든 뒤,  : split
# 각 그룹별 평균을 구해보자    : apply(numeric type), combine

dept_gb = df1.groupby(by='dept')
dept_gb.mean()

/var/folders/3k/bgnrnfc15w792712mgb6g0_c0000gn/T/ipykernel_11792/3270771572.py:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dept_gb.mean()


,salary,age
dept,,
Accounting,4800.0,31.0
Education,4501.2,32.0
Marketing,3735.0,31.5
Sales,8470.0,32.5


In [93]:
# df1을 사용하여 'dept' 별로 group를 만든 뒤,
# 'salary'에 대해서만 각 그룹별 평균을 구해보자

dept_gb = df1.groupby(by='dept')['salary']
dept_gb.mean()

dept
Accounting    4800.0
Education     4501.2
Marketing     3735.0
Sales         8470.0
Name: salary, dtype: float64

In [94]:
# 위의 두 줄을 한 줄로 작성한 것
# df1을 사용하여 'dept' 별로 group를 만든 뒤, 
# 'salary'에 대해서만 각 그룹별 평균을 구해보자

s_dept = df1.groupby(by='dept')['salary'].mean()
s_dept

dept
Accounting    4800.0
Education     4501.2
Marketing     3735.0
Sales         8470.0
Name: salary, dtype: float64

In [95]:
# df1을 사용하여 'dept'별로 'age'에 대한 평균을 구하여 DataFrame으로 작성
df_dept = df1.groupby(by='dept')[['age']].mean()
df_dept

,age
dept,
Accounting,31.0
Education,32.0
Marketing,31.5
Sales,32.5


In [96]:
# df1을 사용하여 'dept'별로 'age', 'salary'에 대해 평균을 구함
df_dept = df1.groupby(by='dept')[['salary', 'age']].mean()
df_dept

,salary,age
dept,,
Accounting,4800.0,31.0
Education,4501.2,32.0
Marketing,3735.0,31.5
Sales,8470.0,32.5


In [97]:
with shelve.open("./data/mysample") as mySample:
    df = mySample['sample4']
    df1 = df.loc[:, ['dept', 'salary', 'chinese', 'gender']]

In [98]:
df1.head()

,dept,salary,chinese,gender
0,Education,3456,0,M
1,Marketing,4320,0,F
2,Education,5600,0,M
3,Education,4500,3,M
4,Marketing,3150,0,F


In [99]:
# 'chinese'의 등급별 개수 세기
df1.groupby(by='chinese')['dept'].count()

chinese
0    5
1    3
2    1
3    1
Name: dept, dtype: int64

In [100]:
# 'dept', 'gender'별 인원 세기  
df1.groupby(by=['dept', 'gender'])['chinese'].count()

dept        gender
Accounting  M         1
Education   F         2
            M         3
Marketing   F         2
Sales       M         2
Name: chinese, dtype: int64

In [101]:
# 'dept', 'gender'별 인원 세기  (value_counts 사용)
df1.groupby(by='dept')['gender'].value_counts()

dept        gender
Accounting  M         1
Education   M         3
            F         2
Marketing   F         2
Sales       M         2
Name: gender, dtype: int64

In [102]:
df3 = df.loc[:, ['dept', 'age', 'gender', 'salary']]

In [103]:
df3.head()

,dept,age,gender,salary
0,Education,34,M,3456
1,Marketing,32,F,4320
2,Education,29,M,5600
3,Education,34,M,4500
4,Marketing,31,F,3150


In [104]:
# 연령대별 'gender'의 인원수 세기
# 연령대를 만드는 작업을 위해 'age'를 index로 변경 한 뒤, 
# 함수를 argument로 하는 groupby사용

df3.set_index('age').groupby(by=lambda x : x//10 * 10)['gender'].value_counts()

age  gender
20   M         1
30   M         5
     F         4
Name: gender, dtype: int64

In [105]:
# 'gender', 'dept'를 index로 설정 후,
# groupby의 level parameter에 [1, 0]을 지정하여 'dept'별, 'gender'별 
# 'salary'의 평균을 구한다

df3.set_index(['gender', 'dept']).groupby(level=[1,0])['salary'].mean()

dept        gender
Accounting  M         4800.000000
Education   F         4475.000000
            M         4518.666667
Marketing   F         3735.000000
Sales       M         8470.000000
Name: salary, dtype: float64

In [106]:
# 'gender', 'dept'를 index로 설정 후,
# groupby의 level parameter에 [0]을 지정하여 'gender'별 
# 'age'의 평균을 구한다

df3.set_index(['gender','dept']).groupby(level=0)['age'].mean()

gender
F    31.500000
M    32.166667
Name: age, dtype: float64

## groupby의 function 관련 메서드 1 - gp.apply, gp.agg

- `gp.apply(func, *args, **kwargs)`
    + 그룹별로 func을 적용하고 결과를 Combine 함
    + func 의 argument : 그룹별 DataFrame/Series 객체
    + 결과로 DataFrame/Series 객체 반환 (변경된 index)
- `gp.agg(func, axis=0, *args, **kwargs)`
    + func : function, str, list or dict
    + func 의 argument : 그룹별 DataFrame/Series 의 각 Series 객체(column)를 인수로 받음
    + 결과로 DataFrame 또는 Series 반환 (변경된 index)

![nn](./images/pandas-08.png)

In [107]:
import warnings   # Suppress Deprecation and Incorrect Usage Warnings
warnings.filterwarnings('ignore')

In [108]:
with shelve.open("./data/mysample") as mySample:
    df = mySample['sample4']
    df1 = df.loc[:, ['dept', 'chinese', 'japanese']]

In [109]:
df1

,dept,chinese,japanese
0,Education,0,1
1,Marketing,0,2
2,Education,0,0
3,Education,3,0
4,Marketing,0,0
5,Education,1,0
6,Accounting,0,0
7,Sales,1,1
8,Sales,2,1
9,Education,1,3


In [110]:
what  = lambda x : (x.shape, type(x))         # 함수의 argument에 대해 shape, type 확인
grade = lambda x : np.where(x>0, 1, 0).sum()  # x에대해 0보다 큰것은 1 아니면 0으로 변경, sum을 적용

s1 = df1.groupby('dept', sort=True).apply(what)
s2 = df1.groupby('dept', sort=True).apply(grade)
s3 = df1.groupby('dept', sort=True).agg([what, grade])  # [what, grade]

display(s1, s2, s3)

dept
Accounting    ((1, 3), <class 'pandas.core.frame.DataFrame'>)
Education     ((5, 3), <class 'pandas.core.frame.DataFrame'>)
Marketing     ((2, 3), <class 'pandas.core.frame.DataFrame'>)
Sales         ((2, 3), <class 'pandas.core.frame.DataFrame'>)
dtype: object

dept
Accounting    0
Education     5
Marketing     1
Sales         4
dtype: int64

chinese             \
                                             <lambda_0> <lambda_1>   
dept                                                                 
Accounting  ((1,), <class 'pandas.core.series.Series'>)          0   
Education   ((5,), <class 'pandas.core.series.Series'>)          3   
Marketing   ((2,), <class 'pandas.core.series.Series'>)          0   
Sales       ((2,), <class 'pandas.core.series.Series'>)          2   

                                               japanese             
                                             <lambda_0> <lambda_1>  
dept                                                                
Accounting  ((1,), <class 'pandas.core.series.Series'>)          0  
Education   ((5,), <class 'pandas.core.series.Series'>)          2  
Marketing   ((2,), <class 'pandas.core.series.Series'>)          1  
Sales       ((2,), <class 'pandas.core.series.Series'>)          2

In [111]:
df2 = df.loc[:, ['dept', 'age', 'salary']]
# 'dept' 별로 age, salary 각각에 대해 min, max값을 구하라  (dept는 정렬할 것)
df.groupby(by='dept', sort=True)['age','salary'].agg([min, max])

age     salary       
           min max    min    max
dept                            
Accounting  31  31   4800   4800
Education   29  34   3456   5600
Marketing   31  32   3150   4320
Sales       32  33   6840  10100

In [112]:
df3 = df.loc[:, ['dept', 'age', 'salary']]
# 'dept'별로 'age'에 대해서는 min, max, 'salary'에 대해서는 np.sum, len, np.mean을 구하라 (dept는 정렬할 것)
# 특정 column에 대해 지정할 때는 dict 객체를 사용한다
df3.groupby(by='dept', sort=True)['age', 'salary'].agg({'age':[min, max], 'salary':[np.sum, len, np.mean]})

age     salary            
           min max    sum len    mean
dept                                 
Accounting  31  31   4800   1  4800.0
Education   29  34  22506   5  4501.2
Marketing   31  32   7470   2  3735.0
Sales       32  33  16940   2  8470.0

## groupby의 function 관련 메서드 2 - transform, filter

- `gp.transform(func)`
    + func 의 argument : 그룹별 DataFrame/Series의 각 Series 객체(column)를 인수로 받음
    + DataFrame 형식을 유지하면서 결과를 구함 (index 유지)
- `gp.filter(func, dropna=True, *args, **kwargs)`
    + 특정 조건으로 데이터를 검색(추출)할 때 사용
    + func 의 return : True/False 를 반환하는 형태여야 함
    + func 의 argument : 그룹별 DataFrame/Series 객체
    + dropna : func 의 결과가 False 인 것에 대해 삭제할 것인가의 여부, dropna = False 인 경우 False 인 것을 NaN 으로 채움
    + filter 된 DataFrame 반환 (변경된 index)

![nn](./images/pandas-09.png)

In [113]:
with shelve.open("./data/mysample") as mySample:
    df = mySample['sample4']

In [114]:
df4 = df.loc[:, ['dept', 'age', 'gender','salary']]
df4.loc[0, 'salary'] = np.nan
df4.head(3)

,dept,age,gender,salary
0,Education,34,M,NaN
1,Marketing,32,F,4320.0
2,Education,29,M,5600.0


In [115]:
df4

,dept,age,gender,salary
0,Education,34,M,NaN
1,Marketing,32,F,4320.0
2,Education,29,M,5600.0
3,Education,34,M,4500.0
4,Marketing,31,F,3150.0
5,Education,32,F,4200.0
6,Accounting,31,M,4800.0
7,Sales,33,M,10100.0
8,Sales,32,M,6840.0
9,Education,31,F,4750.0


In [116]:
what = lambda x : True if type(x) == pd.Series else False  #sum(x)
r1 = df4.groupby('dept').transform(what)
r1
# df4.groupby('dept').transform(lambda x : print(x, end="\n\n"))

,age,gender,salary
0,True,True,True
1,True,True,True
2,True,True,True
3,True,True,True
4,True,True,True
5,True,True,True
6,True,True,True
7,True,True,True
8,True,True,True
9,True,True,True


In [117]:
what1 = lambda x : True if type(x) == pd.DataFrame else False #pd.Series
what2 = lambda x : True if len(x) > 2 else False
# r2 = df4.groupby('dept').filter(what1)
r2 = df4.groupby('dept').filter(what2)
r2

,dept,age,gender,salary
0,Education,34,M,NaN
2,Education,29,M,5600.0
3,Education,34,M,4500.0
5,Education,32,F,4200.0
9,Education,31,F,4750.0


In [118]:
# 'dept', 'gender' 별 'salary'의 평균
df4.groupby(by=['dept', 'gender'])['salary'].mean()

dept        gender
Accounting  M         4800.0
Education   F         4475.0
            M         5050.0
Marketing   F         3735.0
Sales       M         8470.0
Name: salary, dtype: float64

In [119]:
# df4에 's_mean'이라는 열을 추가한다
# 's_mean'은 'dept', 'gender' 별 'salary'의 평균값이다
# 'salary'에서 NA인 것에 대해 df4의 's_mean'에 있는 값으로 변경한다.
df4['s_mean'] = df4.groupby(by=['dept','gender'])['salary'].transform(np.mean)
df4['salary'] = df4['salary'].mask(df4['salary'].isna(), df4['s_mean'])
df4

,dept,age,gender,salary,s_mean
0,Education,34,M,5050.0,5050.0
1,Marketing,32,F,4320.0,3735.0
2,Education,29,M,5600.0,5050.0
3,Education,34,M,4500.0,5050.0
4,Marketing,31,F,3150.0,3735.0
5,Education,32,F,4200.0,4475.0
6,Accounting,31,M,4800.0,4800.0
7,Sales,33,M,10100.0,8470.0
8,Sales,32,M,6840.0,8470.0
9,Education,31,F,4750.0,4475.0


In [120]:
df5 = df.loc[:, ['dept', 'age', 'gender', 'salary']]
# 성별이 남/녀 모두 있는 부서만 추출한다

In [121]:
# df5.groupby(by='dept').filter(lambda x : np.all(x.gender.unique() == ['M', 'F']))
df5.groupby(by='dept').filter(lambda x : np.all(len(x['gender'].unique()) > 1))

,dept,age,gender,salary
0,Education,34,M,3456
2,Education,29,M,5600
3,Education,34,M,4500
5,Education,32,F,4200
9,Education,31,F,4750


# 하나의 열을 여러 행으로 분할하기 - pd.explode

- `pandas.DataFrame.explode`
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html

![nn](./images/pandas-10.png)

In [122]:
import pandas as pd
import numpy as np

In [123]:
def printObj(obj):
    print(type(obj))
    print(obj)
    print("-" * 45)

In [124]:
df = pd.DataFrame({'Menu': ['Drinks','Food','At home Coffee'],
                   'Item': [['Coffees','Teas','Beverages'],
                            ['Bakery','Snacks','Yogurt'],
                            ['Whole Bean','Instant']]})
df

,Menu,Item
0,Drinks,"[Coffees, Teas, Beverages]"
1,Food,"[Bakery, Snacks, Yogurt]"
2,At home Coffee,"[Whole Bean, Instant]"


In [125]:
printObj(df)

<class 'pandas.core.frame.DataFrame'>
             Menu                        Item
0          Drinks  [Coffees, Teas, Beverages]
1            Food    [Bakery, Snacks, Yogurt]
2  At home Coffee       [Whole Bean, Instant]
---------------------------------------------


In [126]:
df = df.explode('Item')
printObj(df)

<class 'pandas.core.frame.DataFrame'>
             Menu        Item
0          Drinks     Coffees
0          Drinks        Teas
0          Drinks   Beverages
1            Food      Bakery
1            Food      Snacks
1            Food      Yogurt
2  At home Coffee  Whole Bean
2  At home Coffee     Instant
---------------------------------------------


In [127]:
df.index = pd.RangeIndex(len(df))
printObj(df)

<class 'pandas.core.frame.DataFrame'>
             Menu        Item
0          Drinks     Coffees
1          Drinks        Teas
2          Drinks   Beverages
3            Food      Bakery
4            Food      Snacks
5            Food      Yogurt
6  At home Coffee  Whole Bean
7  At home Coffee     Instant
---------------------------------------------


In [128]:
midx1 = pd.MultiIndex.from_product([list('AB'), list('ab')])
midx2 = pd.MultiIndex.from_product([list('MF'), list('cd')])
df = pd.DataFrame([[1,[1,2,3],3,4], [9,[2,3],10,11],
                   [4,[1,3],6,7], [1,[7,8,9],10,15]], index=midx1, columns= midx2)

In [129]:
printObj(df)

<class 'pandas.core.frame.DataFrame'>
     M              F    
     c          d   c   d
A a  1  [1, 2, 3]   3   4
  b  9     [2, 3]  10  11
B a  4     [1, 3]   6   7
  b  1  [7, 8, 9]  10  15
---------------------------------------------


In [130]:
df = df.explode(('M', 'd'))
printObj(df)

<class 'pandas.core.frame.DataFrame'>
     M      F    
     c  d   c   d
A a  1  1   3   4
  a  1  2   3   4
  a  1  3   3   4
  b  9  2  10  11
  b  9  3  10  11
B a  4  1   6   7
  a  4  3   6   7
  b  1  7  10  15
  b  1  8  10  15
  b  1  9  10  15
---------------------------------------------
